In [1]:
import numpy as np

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [3]:
identifier = 'Sepsis_WGAN_clip'

In [4]:
real_data_Sepsis = np.load('./data/'+identifier+'real_data.npy')

In [5]:
fake_data_Sepsis = np.load('./data/'+identifier+'fake_data.npy')

In [6]:
real_data_Sepsis.shape

(2627, 30, 5)

In [7]:
fake_data_Sepsis.shape

(2184, 30, 5)

In [8]:
identifier = 'NonSepsis_WGAN_clip'

In [9]:
real_data_NonSepsis = np.load('./data/'+identifier+'real_data.npy')

In [10]:
fake_data_NonSepsis = np.load('./data/'+identifier+'fake_data.npy')

In [11]:
real_data_NonSepsis.shape

(1314, 30, 5)

In [12]:
fake_data_NonSepsis.shape

(1092, 30, 5)

## Train on Synthetic

In [13]:
## 0 - Non-Sepsis
## 1 - Sepsis

In [1]:
#fake_data_NonSepsis[0]

In [2]:
#fake_data_Sepsis[0]

In [16]:
fake_data_X = np.vstack((fake_data_NonSepsis,fake_data_Sepsis))
fake_data_X.shape

(3276, 30, 5)

In [17]:
fake_data_Y = np.concatenate([np.zeros(fake_data_NonSepsis.shape[0]),np.ones(fake_data_Sepsis.shape[0])])
fake_data_Y.shape

(3276,)

In [18]:
seq_len = real_data_Sepsis.shape[1]
feature_len = real_data_Sepsis.shape[2]

In [19]:
print(seq_len)
print(feature_len)

30
5


## Model

In [20]:
import tensorflow as tf

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [22]:
model = Sequential()
model.add(LSTM(100, input_shape=(seq_len, feature_len)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [23]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               42400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 42,501
Trainable params: 42,501
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
from sklearn.model_selection import train_test_split

In [25]:
X_train, X_val, y_train, y_val = train_test_split(fake_data_X, fake_data_Y, test_size=0.2, random_state=42)

In [26]:
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

(2620, 30, 5)
(656, 30, 5)
(2620,)
(656,)


In [27]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10)

Train on 2620 samples, validate on 656 samples
Epoch 1/10
2620/2620 [==============================] - 15s 6ms/step - loss: 0.0535 - acc: 1.0000 - val_loss: 6.0672e-04 - val_acc: 1.0000
Epoch 2/10
2620/2620 [==============================] - 11s 4ms/step - loss: 3.4188e-04 - acc: 1.0000 - val_loss: 2.2090e-04 - val_acc: 1.0000
Epoch 3/10
2620/2620 [==============================] - 12s 5ms/step - loss: 1.5809e-04 - acc: 1.0000 - val_loss: 1.2758e-04 - val_acc: 1.0000
Epoch 4/10
2620/2620 [==============================] - 12s 5ms/step - loss: 1.0009e-04 - acc: 1.0000 - val_loss: 8.6886e-05 - val_acc: 1.0000
Epoch 5/10
2620/2620 [==============================] - 11s 4ms/step - loss: 7.1175e-05 - acc: 1.0000 - val_loss: 6.4626e-05 - val_acc: 1.0000
Epoch 6/10
2620/2620 [==============================] - 13s 5ms/step - loss: 5.4357e-05 - acc: 1.0000 - val_loss: 5.0589e-05 - val_acc: 1.0000
Epoch 7/10
2620/2620 [==============================] - 10s 4ms/step - loss: 4.3759e-05 - acc: 1.00

## Test on Real

In [28]:
real_data_X = np.vstack((real_data_NonSepsis,real_data_Sepsis))
real_data_X.shape

(3941, 30, 5)

In [29]:
real_data_Y = np.concatenate([np.zeros(real_data_NonSepsis.shape[0]),np.ones(real_data_Sepsis.shape[0])])
real_data_Y.shape

(3941,)

In [30]:
scores = model.evaluate(real_data_X, real_data_Y, verbose=1)

3941/3941 [==============================] - 4s 1ms/step


In [31]:
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 52.78%
